In [80]:
import pandas as pd
import xarray as xr
import numpy as np
import scipy.interpolate

import scipy.ndimage as ndimage
%run ../pyfiles/data_cleaning.py

**Table of Contents**
1. <a href=#burned>MCD64A1 Burned Area</a>
2. <a href=#evapo>MOD16A2: EvapoTranspiration Data</a>
3. <a href=#veg>VNP13 Vegetation Indices</a>
4. <a href=#fire>VIIRS VNP14 Thermal Anomaly / Fire</a>
5. <a href=#fwi>Global Fire Weather Database: Fire Weather Index</a>
6. <a href=#merge>Merging Final Dataset</a>

<a id=burned></a>
### MCD64A1 Burned Area
Source: https://lpdaac.usgs.gov/products/mcd64a1v006/

In [50]:
burned = xr.open_mfdataset('../../finalproj_data/time_slice/MCD64A1.006_500m_aid0001.nc', combine = 'by_coords')
burnt = burned.sel(time = slice('2019-11-01','2019-11-01'))

In [51]:
# burnt['crs']
# Attributes:
#     grid_mapping_name:            latitude_longitude
#     _CoordinateAxisTypes:         GeoX GeoY
#     epsg_code:                    4326
#     horizontal_datum_name:        WGS84
#     semi_major_axis:              6378137
#     inverse_flattening:           298.257223563
#     longitude_of_prime_meridian:  0.0

In [52]:
burnt = burnt.drop_vars(['Burn_Date_Uncertainty', 'First_Day', 'Last_Day','crs', 'QA'])
burnt = burnt.squeeze('time')
burnt = burnt.drop('time')

Unique values:
    array([  0.,  nan,  -2., 305., 307., 322., 327., 320., 323., 326., 328.,
       324., 321., 319., 329., 313., 316., 306., 308., 309., 317., 312.,
       311., 333., 334., 332., 331., 310., 325., 318., 314., 315., 330.] <br>
14.6% of 210,892,461 data points are null values. <br>
0 is unburned, -2 is water, other values are days

<a id=evapo></a>
### MOD16A2: EvapoTranspiration Data 

In [84]:
# Load Data

EVAP = xr.open_mfdataset('../../finalproj_data/time_slice/MOD16A2.006_500m_aid0001.nc', combine='by_coords') 
# (lat: 12623, lon: 16707, time: 1)
# 1.687374337 GB
# Source: https://lpdaac.usgs.gov/products/mod16a2v006/
# Probably only going to take Total Evapotranspiration (ET_500m)
# Ignore for now - add if time. 


In [85]:
# <xarray.DataArray 'crs' ()>
# array(-127, dtype=int8)
# Coordinates:
#     time     object 2019-12-11 00:00:00
# Attributes:
#     grid_mapping_name:            latitude_longitude
#     _CoordinateAxisTypes:         GeoX GeoY
#     epsg_code:                    4326
#     horizontal_datum_name:        WGS84
#     semi_major_axis:              6378137
#     inverse_flattening:           298.257223563
#     longitude_of_prime_meridian:  0.0
EVAP = EVAP.drop_vars(['crs', 'ET_QC_500m'])
EVAP = EVAP.squeeze('time')
EVAP = EVAP.drop('time')

In [86]:
EVAP

<xarray.Dataset>
Dimensions:  (lat: 12623, lon: 16707)
Coordinates:
  * lat      (lat) float64 4.36 4.356 4.352 4.348 ... -48.22 -48.23 -48.23
  * lon      (lon) float64 96.33 96.33 96.34 96.34 ... 165.9 165.9 165.9 165.9
Data variables:
    ET_500m  (lat, lon) float32 dask.array<chunksize=(12623, 16707), meta=np.ndarray>
Attributes:
    title:        MOD16A2.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

<a id=veg></a>
### VNP13 Vegetation Indices
Source: https://lpdaac.usgs.gov/products/vnp13a2v001/

In [89]:
VEG = xr.open_mfdataset('../../finalproj_data/time_slice/VNP13A2.001_1km_aid0001.nc', combine = 'by_coords') #  (lat: 6312, lon: 8354, time: 1)
# Slice by time dimension so that this dataset is 1 dimension in time. 
VEG = VEG.sel(time = slice('2019-12-11', '2019-12-11'))
#2.953022425

In [90]:
# TWO
# <xarray.DataArray 'crs' ()>
# array(-127, dtype=int8)
# Attributes:
#     grid_mapping_name:            latitude_longitude
#     _CoordinateAxisTypes:         GeoX GeoY
#     epsg_code:                    4326
#     horizontal_datum_name:        WGS84
#     semi_major_axis:              6378137
#     inverse_flattening:           298.257223563
#     longitude_of_prime_meridian:  0.0

In [91]:
VEG = VEG.drop_vars(['crs'])
VEG = VEG.squeeze('time')
VEG = VEG.drop('time')

In [92]:
VEG

<xarray.Dataset>
Dimensions:                                  (lat: 6312, lon: 8354)
Coordinates:
  * lat                                      (lat) float64 4.362 ... -48.23
  * lon                                      (lon) float64 96.33 96.34 ... 165.9
Data variables:
    _1_km_16_days_EVI                        (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_EVI2                       (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_NDVI                       (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_NIR_reflectance            (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_SWIR1_reflectance          (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_SWIR2_reflectance          (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_SWIR3_reflectance          (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_blue_reflectance           (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_composite_day_of_the_year  (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_green_reflectance          (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_red_reflectance            (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_VI_Quality                 (lat, lon) float64 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
    _1_km_16_days_pixel_reliability          (lat, lon) float32 dask.array<chunksize=(6312, 8354), meta=np.ndarray>
Attributes:
    title:        VNP13A2.001 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

<a id=fire></a>
### VIIRS VNP14 Thermal Anomaly / Fire

Source: https://lpdaac.usgs.gov/products/vnp14a1v001/

In [64]:
THREE = xr.open_mfdataset('../../finalproj_data/time_slice/VNP14A1.001_1km_aid0001.nc', combine = 'by_coords') # (lat: 6312, lon: 8354, time: 1)
#1.054726297 GB


# <xarray.DataArray 'crs' ()>
# array(-127, dtype=int8)
# Attributes:
#     grid_mapping_name:            latitude_longitude
#     _CoordinateAxisTypes:         GeoX GeoY
#     epsg_code:                    4326
#     horizontal_datum_name:        WGS84
#     semi_major_axis:              6378137
#     inverse_flattening:           298.257223563
#     longitude_of_prime_meridian:  0.0

In [65]:
THREE = THREE.drop_vars(['crs', 'QA', 'sample'])
THREE = THREE.squeeze('time')
THREE = THREE.drop('time')

In [66]:
# try dropping null values in xarray
viirs = THREE.dropna(dim = 'lon', subset=['FireMask'])
viirs = viirs.dropna(dim = 'lat', subset=['FireMask'])

In [67]:
# clean xarray dataset
viirs

<xarray.Dataset>
Dimensions:   (lat: 6312, lon: 1899)
Coordinates:
  * lat       (lat) float64 4.362 4.354 4.346 4.337 ... -48.21 -48.22 -48.23
  * lon       (lon) float64 150.1 150.1 150.1 150.1 ... 165.9 165.9 165.9 165.9
Data variables:
    FireMask  (lat, lon) float32 dask.array<chunksize=(6312, 1899), meta=np.ndarray>
    MaxFRP    (lat, lon) float64 dask.array<chunksize=(6312, 1899), meta=np.ndarray>
Attributes:
    title:        VNP14A1.001 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

In [68]:
# # To pandas dataframe

# viirs_df = viirs.to_dataframe()
# viirs_df.reset_index(inplace = True)
# viirs_df['MaxFRP'].fillna(0, inplace = True)

<a id=fwi></a>
### Global Fire Weather Database: Fire Weather Index

In [112]:
# Source: https://data.giss.nasa.gov/impacts/gfwed/
# See data_collection1 for more information. 

# GFWD - FWI --> Dataset labels to identify "high risk of fire" based on FWI calculations.
# Convention to label DataSet in caps and DataArray in lowercase

#Use xarray to open .nc file, combining by coordinates. 
GFWD = xr.open_mfdataset("../../finalproj_data/satellitedata/GFWD/FWI.GEOS-5.Monthly.Default.201912.nc", combine = 'by_coords')

In [113]:
GFWD = GFWD.squeeze('time')
GFWD = GFWD.drop('time')

In [114]:
# Geoslicing based on coordinates for 'burnt'

# lat min : -48.23125 
# lat max: 4.36041667

#lon max = 165.93541665
#lon min = 96.32708332

fwi_ = GFWD.where(GFWD.lat >= -48.23125, drop = True)
fwi_ = fwi_.where(fwi_.lat <= 4.36041667, drop = True)
fwi_ = fwi_.where((fwi_.lon >=96.32708332), drop = True)
fwi = fwi_.where((fwi_.lon <= 165.93541665), drop = True)

<a id=merge></a>

## Merge Datasets

**Downscale MCD64A1 Burned Area (burnt)**

In [76]:
# Reduce dimensionality
# Original: (lat: 12623, lon: 16707)
# Final: (lat: 6312, lon: 1899)
# Scaling Factors (take the inverse)
12623/6312, 16707/1899

(1.9998415716096325, 8.797788309636651)

In [78]:
burnt

<xarray.Dataset>
Dimensions:    (lat: 12623, lon: 16707)
Coordinates:
  * lat        (lat) float64 4.36 4.356 4.352 4.348 ... -48.22 -48.23 -48.23
  * lon        (lon) float64 96.33 96.33 96.34 96.34 ... 165.9 165.9 165.9 165.9
Data variables:
    Burn_Date  (lat, lon) float32 dask.array<chunksize=(12623, 16707), meta=np.ndarray>
Attributes:
    title:        MCD64A1.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

In [81]:
burned_area = scale_variable(burnt, 'Burn_Date', (1/1.9998415716096325, 1/8.797788309636651))

In [83]:
burned_area.shape

(6312, 1899)

**Downscale MOD16A2: EvapoTranspiration Data (EVAP)**

In [43]:
# Original (lat: 12623, lon: 16707)
# Final: (lat: 6312, lon: 1899)
# Scaling Factors (take the inverse)
12623/6312, 16707/1899

(1.9998415716096325, 8.797788309636651)

In [87]:
EVAP
# Var: ET_500m

<xarray.Dataset>
Dimensions:  (lat: 12623, lon: 16707)
Coordinates:
  * lat      (lat) float64 4.36 4.356 4.352 4.348 ... -48.22 -48.23 -48.23
  * lon      (lon) float64 96.33 96.33 96.34 96.34 ... 165.9 165.9 165.9 165.9
Data variables:
    ET_500m  (lat, lon) float32 dask.array<chunksize=(12623, 16707), meta=np.ndarray>
Attributes:
    title:        MOD16A2.006 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

In [88]:
et_500m = scale_variable(EVAP, 'ET_500m', (1/1.9998415716096325, 1/8.797788309636651))

**Downscale VNP13 Vegetation Indices (VEG)**

In [115]:
# Original (lat: 6312, lon: 8354)
# Final: (lat: 6312, lon: 1899)
# Scaling Factors: (take inverse)
6312/6312, 8354/1899

(1.0, 4.399157451290153)

In [109]:
veg_vars = list(VEG.variables)[2:]
veg_vars

['_1_km_16_days_EVI',
 '_1_km_16_days_EVI2',
 '_1_km_16_days_NDVI',
 '_1_km_16_days_NIR_reflectance',
 '_1_km_16_days_SWIR1_reflectance',
 '_1_km_16_days_SWIR2_reflectance',
 '_1_km_16_days_SWIR3_reflectance',
 '_1_km_16_days_blue_reflectance',
 '_1_km_16_days_composite_day_of_the_year',
 '_1_km_16_days_green_reflectance',
 '_1_km_16_days_red_reflectance',
 '_1_km_16_days_VI_Quality',
 '_1_km_16_days_pixel_reliability']

In [111]:
list_veg_arrays = []
for var in veg_vars:
    list_veg_arrays.append(scale_variable(VEG, var, (1, 1/4.399157451290153)))

**Upscale Global Fire Weather Database: Fire Weather Index (fwi)**

In [116]:
# Original: 210 x 222 = 46620
#Final:  6312*1899 = 11986488
# Scaling Factor
6312/210, 1899/222

(30.057142857142857, 8.554054054054054)

In [124]:
# list of all variables that need to be upscaled and appended to viirs xarray
fwi_vars = list(fwi.variables)[:-2]

In [125]:
fwi_arrays = [] #list of output arrays
for name in fwi_vars:
    fwi_arrays.append(scale_variable(fwi, name, (30.057142857142857, 8.554054054054054)))

C:\Users\allis\Anaconda3\envs\merging\lib\site-packages\xarray\coding\variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
C:\Users\allis\Anaconda3\envs\merging\lib\site-packages\xarray\coding\variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
C:\Users\allis\Anaconda3\envs\merging\lib\site-packages\xarray\coding\variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
C:\Users\allis\Anaconda3\envs\merging\lib\site-packages\xarray\coding\variables.py:141: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv
C:\Users\allis\Anaconda3\envs\merging\li

**Merge All Variables to VIIRS**

In [127]:
viirs = viirs.assign(burned = (('lat','lon'),burned_areas))

viirs = viirs.assign(et_500m = (('lat','lon'),et_500m))

viirs = viirs.assign(_1_km_16_days_EVI = (('lat','lon'),list_veg_arrays[0]))
viirs = viirs.assign(_1_km_16_days_EVI2 = (('lat','lon'),list_veg_arrays[1]))
viirs = viirs.assign(_1_km_16_days_NDVI = (('lat','lon'),list_veg_arrays[2]))
viirs = viirs.assign(_1_km_16_days_NIR_reflectance = (('lat','lon'),list_veg_arrays[3]))
viirs = viirs.assign(_1_km_16_days_SWIR1_reflectance = (('lat','lon'),list_veg_arrays[4]))
viirs = viirs.assign(_1_km_16_days_SWIR2_reflectance = (('lat','lon'),list_veg_arrays[5]))
viirs = viirs.assign(_1_km_16_days_SWIR3_reflectance = (('lat','lon'),list_veg_arrays[6]))
viirs = viirs.assign(_1_km_16_days_blue_reflectance = (('lat','lon'),list_veg_arrays[7]))
viirs = viirs.assign(_1_km_16_days_composite_day_of_the_year = (('lat','lon'),list_veg_arrays[8]))
viirs = viirs.assign(_1_km_16_days_green_reflectance = (('lat','lon'),list_veg_arrays[9]))
viirs = viirs.assign(_1_km_16_days_red_reflectance = (('lat','lon'),list_veg_arrays[10]))
viirs = viirs.assign(_1_km_16_days_VI_Quality = (('lat','lon'),list_veg_arrays[11]))
viirs = viirs.assign(_1_km_16_days_pixel_reliability = (('lat','lon'),list_veg_arrays[12]))

viirs = viirs.assign(_DC = (('lat','lon'),fwi_arrays[0]))
viirs = viirs.assign(_DMC= (('lat','lon'),fwi_arrays[1]))
viirs = viirs.assign(_FFMC= (('lat','lon'),fwi_arrays[2]))
viirs = viirs.assign(_ISI= (('lat','lon'),fwi_arrays[3]))
viirs = viirs.assign(_BUI= (('lat','lon'),fwi_arrays[4]))
viirs = viirs.assign(_FWI= (('lat','lon'),fwi_arrays[5]))
viirs = viirs.assign(_DSR= (('lat','lon'),fwi_arrays[6]))

In [128]:
viirs

<xarray.Dataset>
Dimensions:                                  (lat: 6312, lon: 1899)
Coordinates:
  * lat                                      (lat) float64 4.362 ... -48.23
  * lon                                      (lon) float64 150.1 150.1 ... 165.9
Data variables:
    FireMask                                 (lat, lon) float32 dask.array<chunksize=(6312, 1899), meta=np.ndarray>
    MaxFRP                                   (lat, lon) float64 dask.array<chunksize=(6312, 1899), meta=np.ndarray>
    burned                                   (lat, lon) float32 0.0 0.0 ... -2.0
    et_500m                                  (lat, lon) float32 3276.5 ... 3276.6
    _1_km_16_days_EVI                        (lat, lon) float32 0.611 ... nan
    _1_km_16_days_EVI2                       (lat, lon) float32 0.6071 ... nan
    _1_km_16_days_NDVI                       (lat, lon) float32 0.84599996 ... nan
    _1_km_16_days_NIR_reflectance            (lat, lon) float32 0.3885 ... nan
    _1_km_16_days_SWIR1_reflectance          (lat, lon) float32 0.34899998 ... nan
    _1_km_16_days_SWIR2_reflectance          (lat, lon) float32 0.1735 ... nan
    _1_km_16_days_SWIR3_reflectance          (lat, lon) float32 0.073199995 ... nan
    _1_km_16_days_blue_reflectance           (lat, lon) float32 0.0168 ... nan
    _1_km_16_days_composite_day_of_the_year  (lat, lon) float32 360.0 ... nan
    _1_km_16_days_green_reflectance          (lat, lon) float32 0.051799998 ... nan
    _1_km_16_days_red_reflectance            (lat, lon) float32 0.0324 ... nan
    _1_km_16_days_VI_Quality                 (lat, lon) float64 2.257e+03 ... 5.734e+04
    _1_km_16_days_pixel_reliability          (lat, lon) float32 6.0 7.0 ... nan
    _DC                                      (lat, lon) float32 nan nan ... nan
    _DMC                                     (lat, lon) float32 nan nan ... nan
    _FFMC                                    (lat, lon) float32 nan nan ... nan
    _ISI                                     (lat, lon) float32 nan nan ... nan
    _BUI                                     (lat, lon) float32 nan nan ... nan
    _FWI                                     (lat, lon) float32 nan nan ... nan
    _DSR                                     (lat, lon) float32 nan nan ... nan
Attributes:
    title:        VNP14A1.001 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v2.34
    references:   See README.txt
    history:      See README.txt

In [129]:
viirs.to_netcdf('../../finalproj_data/satellitedata/final_merge.nc')

### Create a Labels Column in the Cloud (using pandas)

<img src='../images/fire_danger.png'>

In [90]:
merged['fire_weather'] = 0

In [91]:
# Create categorical values from FWI numerical
merged.loc[(merged['FWI'] < 5), 'fire_weather'] = 'fwi_low'
merged.loc[(merged['FWI'] > 5) & (merged['FWI'] < 8), 'fire_weather'] = 'fwi_moderate'
merged.loc[(merged['FWI'] > 8) & (merged['FWI'] < 16), 'fire_weather'] = 'fwi_high'
merged.loc[(merged['FWI'] > 16) & (merged['FWI'] < 29), 'fire_weather'] = 'fwi_veryhigh'
merged.loc[(merged['FWI'] > 29), 'fire_weather'] = 'fwi_extreme'
merged.drop('FWI', axis = 1, inplace = True)

In [93]:
merged['label'] = 0

In [ ]:
## You will probably have to run this part in the cloud

In [96]:
for row in range(0, len(merged)):
    if merged.loc[row,'fire_weather'] == 0:
        merged.loc[row,'label'] = merged.loc[row,'FireMask']
    else:
        None

KeyboardInterrupt: 